In [7]:
import os
import numpy as np
from osgeo import gdal
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb

/home/patras/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/patras/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [8]:
P = 3
Q = 3
R = 1

cellsize = 250.0

In [9]:
# input path (.grb)
path_raw = '../../data/rawdata/COSMO6_EU_201908/'
# intermediate path (.nc)
path_prep = '../../data/prepareddata/COSMO6_VM_201908/'
# output path (.pfb)
path_pfb = '../../data/pfdata/'

version = '1' # processing version matching gridtype_valmalencodomain_v<version>.txt file
versiondist = 'v1-331' # pfb distribution

In [10]:
# equivalent names btw : COSMO_REA6 filename, NLDAS, COSMO_REA6 internal file name (?)

variables = [['ASWDIFD_S', 'DSWR', 'ASWDIFD_S'],
             ['ATHD_S', 'DLWR', 'ATHD_S'],
             ['PS', 'Press', 'sp'],
             ['QV_S', 'SPFH', 'QV_S'],
             ['T_2M', 'Temp', '2t'],
             ['TOT_PRECIP', 'APCP', 'tp'],
             ['U_10M', 'UGRD', '10u'],
             ['V_10M', 'VGRD', '10v']]

month = '08'
day1 = 25
day2 = 31
year = '2019'

In [11]:
for variable in variables:
    filename_org = variable[0] +'.'+year+month
    filename = variable[0]+'.v'+version +'.'+year+month # clipped
    os.system('cdo -f nc copy ' + path_raw + variable[0] +'.2D.'+year+month+'.grb ' + path_prep + filename_org +'.nc')
    os.system('ncrename -v '+variable[2]+','+variable[1] + ' ' + path_prep + filename_org +'.nc')
    os.system('cdo -remapbil,../../data/gridtype_valmalencodomain_v'+version+'.txt '+ path_prep + filename_org + '.nc ' + path_prep + filename +'.nc')
    if variable[0] == 'TOT_PRECIP':
        #unit conversion (mm/s to kg/m^2)
        os.system('cdo divc,3600 '+ path_prep + filename +'.nc '+ path_prep + filename +'cv.nc')
        os.system('rm '+ path_prep + filename + '.nc')
        os.system('mv '+ path_prep + filename +'cv.nc ' + path_prep + filename +'.nc')
    
    for day in range(day1,day2+1):
        daily_filename = path_prep + filename+str(day).zfill(2) +'.nc'
        hours_range = str((day-1)*24+1) + '/' + str(day*24) + ' '
        os.system('cdo seltimestep,'+ hours_range + path_prep + filename +'.nc '+ daily_filename)
        daily_dataset = gdal.Open("NETCDF:{0}:{1}".format(daily_filename, variable[1]))

        nt = daily_dataset.RasterCount
        nx = daily_dataset.RasterXSize
        ny = daily_dataset.RasterYSize
        clm_daily = np.empty((nt, ny, nx), dtype=np.float64)
        buf_obj = np.empty((ny, nx), dtype=np.float64)
        bands_data = []
        for hour in range(24, 0, -1): #invert order for pfb
            band_data = daily_dataset.GetRasterBand(hour) # band = hour
            clm_daily[24-hour] = np.array(band_data.ReadAsArray(buf_obj = buf_obj))
        
        clm_daily = clm_daily.astype(np.float64)
        pfb_name = path_pfb +'clm.'+variable[1]+'.'+versiondist+'.'+year+month+str(day).zfill(2)+'.pfb'
        write_pfb( get_absolute_path(pfb_name), clm_daily, p=P, q=Q, r=R, x=0, y=0, z=0, dx=cellsize, dy=cellsize, dz=1.0, dist = False)

Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation fault
Segmentation

# END

In [25]:
# print(np.__version__) : 1.26.4 - downgraded to make compatible with gdal

1.26.4
